In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
complete_path = os.getcwd()
if 'nb' in complete_path:
    os.chdir("..")

In [3]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib
import os
import pandas as pd
import yaml
from IPython.display import clear_output
import warnings

In [4]:
from sklearn.kernel_ridge import KernelRidge
import time
from mlnrg.utils.logger import log

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels

In [5]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
from mlnrg.utils.mpl_at_utils import MPLAdjutant
from mlnrg.loader import NRGData
from mlnrg.utils import defaults

In [6]:
from mlnrg.utils import analysis_tools as at

In [8]:
!ls -lthr results/DC-KRR/*kondo*

-rw-r--r--  1 erica  staff   1.0G Sep 12 18:45 results/DC-KRR/kondo_full_alpha0.01_gamma1_subsets8.pkl
-rw-r--r--  1 erica  staff   1.0G Sep 12 23:04 results/DC-KRR/kondo_full_alpha0_gamma1_subsets8.pkl
-rw-r--r--  1 erica  staff   1.0G Sep 16 22:22 results/DC-KRR/kondo_full_alpha0.01_gamma0.01_subsets8.pkl
-rw-r--r--  1 erica  staff   1.0G Sep 20 23:51 results/DC-KRR/kondo_full_alpha0_gamma0.01_subsets8.pkl
-rw-r--r--  1 erica  staff   169M Sep 21 18:39 results/DC-KRR/KRR_kondo_fps_alpha0_gamma1_subset1_comparison.pkl
-rw-r--r--  1 erica  staff    46M Oct  1 14:23 results/DC-KRR/FINAL_kondo_full_alpha0_gamma1_subsets8.pkl


In [9]:
file_name = open('results/DC-KRR/kondo_full_alpha0_gamma1_subsets8.pkl', 'rb')
results = pickle.load(file_name)
results.keys()

dict_keys(['set', 'sampling', 'tr_set_size', 'number_subsets', 'val_set_size', 'trained_weights', 'fit_time', 'sorted_errors', 'costs'])

In [ ]:
print(results['set'])
print(results['sampling'])
print(results['tr_set_size'])
print(results['number_subsets'])
set_size = float(results['tr_set_size'])/float(results['number_subsets'])
print(set_size)
print(results['fit_time'])
print(results['costs'])

In [ ]:
avg_tr_score = np.mean(np.array(results['costs'])[:,0])
avg_val_score = np.mean(np.array(results['costs'][:,1]))

log.info("\nThe average training r^2 score:\t\t%.5g\nThe average validation r^2 score:\t%.5g"
        %(avg_tr_score, avg_val_score)
        )

In [ ]:
validation_error_array = np.array(results['sorted_errors'])
raw_val_errors = validation_error_array[:,3]

mean_MAE_val = np.mean(raw_val_errors)
std_MAE_val  = np.std(raw_val_errors)

print(f"Avg validation err: %.5f +/- %.5f" % (mean_MAE_val, std_MAE_val))
print(f"log10 Avg validation err: %.5f +/- %.5f" % (np.log10(mean_MAE_val), np.log10(std_MAE_val)))
print(f"log10 validation median: %.5f" % np.log10(np.median(raw_val_errors)))

In [ ]:
validation_errors = []
for i  in range(0, len(raw_val_errors)):
    validation_errors.append(np.log10(raw_val_errors[i]))

plt.clf()

y_Height = 0.05 * len(validation_errors)
mean = np.log10(np.mean(raw_val_errors))
median = np.log10(np.median(raw_val_errors))

plt.hist(validation_errors, bins=60, color='k')
for ii in range(1, 10):
    x = np.percentile(validation_errors, ii*10)
    plt.plot((x, x), (0, y_Height), color='r', label="Deciles" if ii == 1 else None)
plt.plot((mean, mean), (0, y_Height), color='orange', label=f"Mean: {mean:.05g}")
plt.plot((median, median), (0, y_Height), color='b', label=f"Median: {median:.05g}")

plt.title("Kondo Validation Trials KRR MAE")
plt.xlabel("$log_{10}(\mathrm{MAE})$")
plt.ylabel("Occurances")
plt.legend(loc="upper left", frameon=False)

#plt.savefig("Hist_anderson_fps_a" + str(alpha) + "_g" + str(gamma) +"_errors.png", dpi=300, bbox_inches='tight')
plt.show()

## And now, for the TEST set!!! Do not run until after hyperparameter tuning :)

In [ ]:
original_data_set = pd.read_csv('data/Kondo_411267_trials_with_headers.csv')
split_indexes = pickle.load(open("results/fps_files/kondo_fps.pkl", "rb"))
#original_data_set = pd.read_csv('data/Anderson_599578_trials_with_headers.csv')
#split_indexes = pickle.load(open("results/fps_files/anderson_fps.pkl", "rb"))

training_idx = list(split_indexes[0]['train'])
validation_idx = list(split_indexes[0]['valid'])
test_idx = list(split_indexes[0]['test'])

In [ ]:
# # Start with B: rescale with symlog after adjusting for B=0 cases.
# m = (original_data_set[original_data_set['B'] != 0]['B']).abs().min()
# original_data_set.loc[list((original_data_set[original_data_set['B'] == 0]).index), 'B'] = m/10

# sign_B = np.sign(original_data_set['B'])
# original_data_set.loc[:, 'B'] = sign_B * np.log10(np.abs(original_data_set.loc[:, 'B']))
# #data_set.rename(columns={'B': 'symlog10 B'}, inplace=True)

# # Now do T: rescale with regular log after adjusting for T=0 cases.
# m = (original_data_set[original_data_set['T'] != 0]['T']).abs().min()
# original_data_set.loc[list((original_data_set[original_data_set['T'] == 0]).index), 'T'] = m/10
# original_data_set.loc[:, 'T'] = np.log10(original_data_set.loc[:, 'T'])
# #data_set.rename(columns={'T': 'log10 T'}, inplace=True)

# # Now do Gamma: recale with regular log.
# original_data_set.loc[:, 'Gamma'] = np.log10(original_data_set.loc[:, 'Gamma'])
# #data_set.rename(columns={'Gamma': 'log10 Gamma'}, inplace=True)

In [ ]:
unscaled_tr_features = (original_data_set.loc[training_idx[:],'U':'T']).to_numpy()
tr_targets = (original_data_set.iloc[training_idx[:], 8:]).to_numpy()
number_subsets = results['number_subsets']
remainder_trials = len(unscaled_tr_features) % number_subsets
log.info("\nTo attain the desired number of subsets, %d trials had to be removed from training." 
         % remainder_trials
        )

if (remainder_trials >= len(unscaled_tr_features)*0.001):
    log.warn("The number of subsets requested will require the removal of %d, "\
             "more than 1/1000 of the available training data." 
            )
if (remainder_trials != 0):    
    unscaled_tr_features = unscaled_tr_features[:-1*remainder_trials]
    tr_targets = tr_targets[:-1*remainder_trials]
    
subset_size = len(unscaled_tr_features)/number_subsets # (Will be 5K if using all default values.)

all_starting_trials = len(unscaled_tr_features)
log.info("\nOriginal number of training trials:\t%d\nRemaining number of training trials:\t%d"\
        "\nRemoved trials:\t\t%d (%.5g%% of the total trials)\nNumber of subsets:\t%d\n"\
        "Trials per subset:\t%6.1f"
         % (all_starting_trials, len(unscaled_tr_features), remainder_trials, 
            (100*remainder_trials/all_starting_trials), number_subsets, subset_size) 
        )

subset_size = int(subset_size)

In [ ]:
unscaled_TEST_features = (original_data_set.loc[test_idx,'U':'T']).to_numpy()
TEST_targets = (original_data_set.iloc[test_idx[:], 8:]).to_numpy()
TEST_indexes = (original_data_set.loc[test_idx,'idx']).to_numpy()

In [ ]:
mu_scaling = unscaled_tr_features.mean(axis=0)
std_scaling = unscaled_tr_features.std(axis=0)

tr_features = (unscaled_tr_features - mu_scaling) /std_scaling
TEST_features = (unscaled_TEST_features - mu_scaling) / std_scaling

In [ ]:
alpha = 0
gamma = 1

trained_weights = np.array(results['trained_weights'])
all_tr_subset_features = np.array_split(tr_features, number_subsets) 
all_tr_subset_targets = np.array_split(tr_targets, number_subsets)
all_alpha_weights = results['trained_weights']

sum_predictions = np.zeros([1, TEST_targets.shape[1]]) 
TEST_costs = np.empty([number_subsets, 1])

time_start = time.time()
for s in range(0, number_subsets):
    t0 = time.time()
    iter_tr_features = all_tr_subset_features[s]
    iter_tr_weight = np.reshape(all_alpha_weights[s], [int(set_size), TEST_targets.shape[1]])
    
    iter_kernel_of_TEST_and_tr = pairwise_kernels(TEST_features, iter_tr_features, 
                                                  metric='laplacian', 
                                                  alpha=alpha,
                                                  gamma=gamma, 
                                                  filter_params=True
                                                 )
    
    iter_prediction = np.dot(iter_kernel_of_TEST_and_tr, iter_tr_weight)
    
    u_value = ((TEST_targets - iter_prediction)**2).sum()
    v_value = ((TEST_targets - TEST_targets.mean())**2).sum()
    iter_TEST_cost = 1 - (u_value/v_value)
    TEST_costs[s,0] = iter_TEST_cost
    
    sum_predictions = np.add(sum_predictions, iter_prediction)
    iter_time = time.time() - t0
    
    print("Iteration %d/%d compelte in: %.3g seconds\nTesting Cost = %.5g\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" 
          % (s+1, number_subsets, iter_time, iter_TEST_cost)
         )
    
#--------------------------------
# THIS IS THE FINAL PREDICTION!!!
#--------------------------------
averaged_TEST_predictions = sum_predictions / number_subsets

fit_time = time.time() - time_start
log.info("\nAll %d/%d subsets with %d trials/subset tested and averaged.\n %d trials in test set. Total time: %.5g seconds" 
         % (s+1, number_subsets, subset_size, len(TEST_targets), fit_time)
        )   

In [ ]:
from collections import namedtuple
Result = namedtuple('Result', ['pred', 'target', 'feature', 'meta', 'name', 'mae'])

In [ ]:
raw_TEST_errors = np.mean(np.abs(TEST_targets - averaged_TEST_predictions), axis=1)
sorted_TEST_errors = []
all_test_results = []
for ii in range(len(raw_TEST_errors)):
  sorted_TEST_errors.append([ii, TEST_targets[ii], averaged_TEST_predictions[ii], raw_TEST_errors[ii]])
  all_test_results.append(Result(pred=averaged_TEST_predictions[ii].tolist(),
                                 target=TEST_targets[ii].tolist(), 
                                 feature=TEST_features[ii].tolist(), 
                                 meta=None, 
                                 name=int(TEST_indexes[ii]), 
                                 mae=float(raw_TEST_errors[ii])
                                )
                         )

sorted_TEST_errors.sort(key=lambda x: x[3])

In [ ]:
plt.clf()

y_Height = 0.05 * len(raw_TEST_errors)
mean = np.log10(np.mean(raw_TEST_errors))
median = np.log10(np.median(raw_TEST_errors))

plt.hist(np.log10(raw_TEST_errors), bins=60, color='k')
for ii in range(1, 10):
    x = np.percentile(np.log10(raw_TEST_errors), ii*10)
    plt.plot((x, x), (0, y_Height), color='r', label="Deciles" if ii == 1 else None)
plt.plot((mean, mean), (0, y_Height), color='orange', label=f"Mean: {mean:.05g}")
plt.plot((median, median), (0, y_Height), color='b', label=f"Median: {median:.05g}")

plt.title("FPS Kondo Test Trials KRR MAE")
plt.xlabel("$log_{10}(\mathrm{MAE})$")
plt.ylabel("Occurances")
plt.legend(loc="upper left", frameon=False)

#plt.savefig("Hist_anderson_fps_a" + str(alpha) + "_g" + str(gamma) +"_errors.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("All test r^2 values:")
print(TEST_costs)
print("\nAveraged test r^2 value: %.8f\n" % np.mean(TEST_costs))

mean_MAE_TEST_set = np.mean(raw_TEST_errors)
std_MAE_TEST_set = np.std(raw_TEST_errors)
median_MAE_TEST_set = np.median(raw_TEST_errors)
print(f"Avg  err: %.5f +/- %.5f" % (mean_MAE_TEST_set, std_MAE_TEST_set))

print(f"Log10 mean MAE:\t %.5f +/- %.5f" % (np.log10(mean_MAE_TEST_set), np.log10(std_MAE_TEST_set)))
print(f"Log10 median MAE:\t %.5f" % np.log10(median_MAE_TEST_set))

In [ ]:
for i in range(0,len(TEST_costs)):
    print(i)
    print(np.array([results['costs'][i,1]]))
    print(TEST_costs[i])
    print("--------------------")

In [ ]:
plt.clf()

fig = plt.figure(figsize=(4,3))

plt.scatter(np.arange(1,number_subsets+1), np.array(results['costs'][:,1]), 
            color="black", s=10,
            label="Validation")
plt.scatter(np.arange(1,number_subsets+1), TEST_costs, 
            facecolors='none', edgecolors='red', s=10,
            label="Test")  

plt.xticks(np.arange(1,8.9), fontsize=8)
plt.yticks(np.arange(0.5,1.1,.1), fontsize=8)
plt.xlim([0.3,8.65])
plt.ylim([0.45, 1.025])
plt.legend(loc="lower left", prop={'size': 6}, frameon=False)
plt.xlabel("Subset $s$", fontsize=8)
plt.ylabel(r"$r^2$ score",fontsize=8)



plt.show()

### Save relevant info and pickle it

In [ ]:
# Put all r^2 scores into a single dataframe. 
all_r2_scores = pd.DataFrame(
    {
        'train': results['costs'][:,0],
        'valid': results['costs'][:,1],
        'test': TEST_costs[:,0]
    }
)
print(all_r2_scores)

In [ ]:
# Set up information for pickle in a dictionary because it's convenient!

dckrr_pickle_info = {
    "dataset": "kondo",
    "N_total": 411267,
    "S_subsets": number_subsets,
    "n_trials_per_subset": set_size,
    "ker_params": {"kernel": 'laplacian', "alpha": alpha, "gamma": gamma},
    "sampler": "fps",
    "normalize_features_via_train": True,
    "scale_features_via_train": False,
    "r2_scores": all_r2_scores,
    "RESULTS": all_test_results 
}


pickle.dump(dckrr_pickle_info, open("results/DC-KRR/FINAL_kondo_full_alpha0_gamma1_subsets8.pkl", "wb" ))


# dckrr_info = {
#     "set": "anderson",
#     "sampling": "fps",
#     "alpha": alpha,
#     "gamma": gamma,
#     "tr_set_size": len(tr_features),
#     "number_subsets": number_subsets,
#     "val_set_size": len(val_features),
#     "trained_weights": alpha_weights,
#     "fit_time": fit_time,
#     "sorted_errors": sorted_errors,
#     "costs": iter_costs
# }



# {'batch_sizes': [1000, 1000, 5120],
#  'dataset': 'anderson',
#  'dataset_info': {'ntestval': 24000,
#   'ntotal': 599578,
#   'path': '/hpcgpfs01/scratch/mcarbone/mlnrg/Anderson_599578_trials_with_headers.csv'},
#  'epochs': 1000,
#  'grid_path': '/hpcgpfs01/scratch/mcarbone/mlnrg/Omega_grid_values.csv',
#  'idx_dir': '.idx',
#  'loss': 'l1',
#  'net_params': {'dropout': 0.0, 'hidden_size': 100, 'n_hidden_layers': 3},
#  'protocol': 'standardMLP',
#  'sampler': 'fps',
#  'sampler_type': None,
#  'scale_features_via_train': True,
#  'scale_properties': True,
#  'target_dir': '/sdcc/u/mcarbone/mlnrg/results/downsampling_check_rerun/004',
#  'train_up_to': 50000,
#  'transform': None}

### And now for analysis

In [ ]:
dataset_map = {
    'kondo': 'data/Kondo_411267_trials_with_headers.csv',
    'anderson': 'data/Anderson_599578_trials_with_headers.csv',
    'grid': 'data/Omega_grid_values.csv'
    
}

In [ ]:
data_class_no_scale = NRGData()
data_class_no_scale.load(path=dataset_map['anderson'])

In [ ]:
test_results, test_unscaled_energies = at.set_up_df(all_test_results, data_class_no_scale, "anderson")

In [ ]:
deciles, mean, median = at.plot_err_histogram(test_results['err'], 'test')

In [ ]:
at.plot_param_best_and_worst_dist(test_results, 1000)

In [ ]:
at.plot_best_and_worst_SPE_control(test_results, test_unscaled_energies, "anderson")

In [ ]:
at.plot_percentile(all_test_results, data_class_no_scale, percentile=50)

In [ ]:
at.plot_single_prediction(all_test_results, data_class_no_scale, np.array(data_class_no_scale.grid), 50000)

In [ ]:
at.plot_single_trial(data_class_no_scale, np.array(data_class_no_scale.grid), 91412)

In [ ]:
#np.random.seed(seed=42)
test_names = (original_data_set.loc[test_idx[:],'idx']).to_numpy()

selections = np.random.choice(test_names, size=25, replace=False)
print(selections)
at.plot_random_25(all_test_results, data_class_no_scale, np.array(data_class_no_scale.grid), selections)

In [ ]:
for x,y in zip(np.arange(1,number_subsets+1), np.array(results['costs'][:,1])):
    label1 = "{:.4f}".format(y)

    plt.annotate(label1, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,-8), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0
                )
for xx,yy in zip(np.arange(1,10), np.array(TEST_costs.flatten())[:10]):
    label2 = "{:.4f}".format(yy)

    plt.annotate(label2, # this is the text
                 (xx,yy), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,5), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0,
                 color="red"
                )  
    
    
    
    ############################### BEGIN NOTATION  ############################
for x,y in zip(np.array([1,2,3,4,5,6,8,9]), np.array(results['costs'][[0,1,2,3,4,5,7,8,9],1])):
    label1 = "{:.4f}".format(y)

    plt.annotate(label1, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,-8), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0
                )
for xx,yy in zip(np.array([1,2,3,4,5,6,8,9]), np.array(TEST_costs.flatten())[[0,1,2,3,4,5,7,8]]):
    label2 = "{:.4f}".format(yy)

    plt.annotate(label2, # this is the text
                 (xx,yy), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,5), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0,
                 color="red"
                )    
############################### POINTS 10,11 ANDERSON ############################    
for x,y in zip(np.arange(9,number_subsets-1), np.array(results['costs'][9:11,1])):
    label1 = "{:.4f}".format(y)

    plt.annotate(label1, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(10,5), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0
                )
for xx,yy in zip(np.arange(9, number_subsets-1), np.array(TEST_costs.flatten())[9:11]):
    label2 = "{:.4f}".format(yy)

    plt.annotate(label2, # this is the text
                 (xx,yy), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(10,-8), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0,
                 color="red"
                )  
############################### LAST POINT ANDERSON ############################    
for x,y in zip(np.array([7]), np.array([results['costs'][6,1]])):
    label1 = "{:.4f}".format(y)

    plt.annotate(label1, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,5), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0
                )
for xx,yy in zip(np.array([7]), TEST_costs[6]):
    label2 = "{:.4f}".format(yy)

    plt.annotate(label2, # this is the text
                 (xx,yy), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,-8), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0,
                 color="red"
                )  

for x,y in zip(np.array([12]), np.array([results['costs'][11,1]])):
    label1 = "{:.4f}".format(y)

    plt.annotate(label1, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,-8), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0
                )
for xx,yy in zip(np.array([12]), TEST_costs[11]):
    label2 = "{:.4f}".format(yy)

    plt.annotate(label2, # this is the text
                 (xx,yy), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-8,5), # distance from text to points (x,y)
                 ha='left', # horizontal alignment can be left, right or center
                 fontsize=4.5,
                 rotation=0,
                 color="red"
                )   
###########################################################################  